In [14]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torchvision import datasets, transforms
from functools import partial

In [20]:


class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.mnist_model = nn.Sequential(
            nn.Conv2d(1, 8, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Dropout(0.1),
            nn.Conv2d(8, 12, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(12),
            nn.Dropout(0.1),
            nn.Conv2d(12, 16, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),
            nn.Conv2d(16, 32, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            nn.Conv2d(32, 16, 1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),
            nn.Conv2d(16, 16, 3, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),
            nn.Conv2d(16,16,3,bias = False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.25),
            nn.Conv2d(16,10,3)
            
        )
        #self.init_weight()
        
    def forward(self, x):
        x = self.mnist_model(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)
    


In [21]:
torch.manual_seed(1)
batch_size = 128 

kwargs = {'num_workers': 1, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        
        transforms.Normalize((0.1307, ), (0.3081, ))
    ])), batch_size=batch_size, shuffle=True,**kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, 
    transform=transforms.Compose([
        transforms.ToTensor(),
        
        transforms.Normalize((0.1307, ), (0.3081, ))
    ])), batch_size=batch_size, shuffle=True,**kwargs)

In [22]:
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)
def train(model, device, train_loader, optimizer, epoch):
    train_correct = 0
    train_loss = 0
    model.train()
    for _, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.view_as(pred)).sum().item()
        loss = F.nll_loss(output, target)
        train_loss += loss
        loss.backward()
        optimizer.step()
        for param in optimizer.param_groups:
          lr = param['lr']
    train_loss /= len(train_loader.dataset)

    
    #return 100.0 * train_correct/len(train_loader.dataset)
    print (f'\nTrain set: Average loss: {train_loss}, Train Accuracy: {train_correct}/{len(train_loader.dataset)} ({100. * train_correct / len(train_loader.dataset)}), learning rate : {lr}')
    
    
    

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for _, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

   # return  1.0 * correct / len(test_loader.dataset)

    print (f'\nTest set: Average loss: {test_loss}, Test Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset)})')

In [23]:
device = torch.device('cuda')
model = MNISTNet().to(device)
summary(model, input_size=(1, 28, 28))

init_lr = 0.03
optimizer = optim.Adam(model.parameters(), lr = init_lr)
def adjust_lr(optimizer, epoch):
    for param_group in optimizer.param_groups:
      init_lr = param_group['lr']
    lr = max(round(init_lr * 1/(1 + np.pi/50 * epoch), 10), 0.0005)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 12, 24, 24]             864
              ReLU-6           [-1, 12, 24, 24]               0
       BatchNorm2d-7           [-1, 12, 24, 24]              24
           Dropout-8           [-1, 12, 24, 24]               0
            Conv2d-9           [-1, 16, 22, 22]           1,728
             ReLU-10           [-1, 16, 22, 22]               0
      BatchNorm2d-11           [-1, 16, 22, 22]              32
          Dropout-12           [-1, 16, 22, 22]               0
           Conv2d-13           [-1, 32, 20, 20]           4,608
             ReLU-14           [-1, 32,

In [24]:
for epoch in range(20):
    print(f'EPOCH {epoch + 1} / {20}')
    adjust_lr(optimizer,epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH 1 / 20

Train set: Average loss: 0.0014930437318980694, Train Accuracy: 56440/60000 (94.06666666666666), learning rate : 0.03

Test set: Average loss: 0.05383644685745239, Test Accuracy: 9831/10000 (98.31)
EPOCH 2 / 20

Train set: Average loss: 0.0006989385583437979, Train Accuracy: 58357/60000 (97.26166666666667), learning rate : 0.0282264781

Test set: Average loss: 0.06262335634231567, Test Accuracy: 9817/10000 (98.17)
EPOCH 3 / 20

Train set: Average loss: 0.0005632475367747247, Train Accuracy: 58638/60000 (97.73), learning rate : 0.0250754092

Test set: Average loss: 0.03351845169067383, Test Accuracy: 9891/10000 (98.91)
EPOCH 4 / 20

Train set: Average loss: 0.0004940114449709654, Train Accuracy: 58798/60000 (97.99666666666667), learning rate : 0.0210984459

Test set: Average loss: 0.0333072110414505, Test Accuracy: 9899/10000 (98.99)
EPOCH 5 / 20

Train set: Average loss: 0.0004347472277004272, Train Accuracy: 58949/60000 (98.24833333333333), learning rate : 0.0168608517

